# Theory stuff

## Import things

In [1]:
import numpy as np
import scipy as sp
from scipy import linalg
from numpy import linalg,random
import matplotlib.pyplot as plt
from matplotlib import cm,ticker,colors,rc,font_manager
%matplotlib inline

## Quantum Information Definitions
Let's start by defining some functions for the most important operators/states

In [2]:
#convert deg to rad
def theta(x):
    return x*np.pi/180

# Lambda-Half wave plate
def HWP(x):
    return np.array([[np.cos(2*theta(x)),np.sin(2*theta(x))],[np.sin(2*theta(x)),-np.cos(2*theta(x))]])

# QWP
def QWP(x):
    return np.array([[np.cos(theta(x))**2+1j*np.sin(theta(x))**2,(1-1j)*np.sin(theta(x))*np.cos(theta(x))],
            [(1-1j)*np.sin(theta(x))*np.cos(theta(x)),np.sin(theta(x))**2+1j*np.cos(theta(x))**2]])

#identity matrix
idn = np.eye(2)

#basic states
h = np.array([[1],[0]])
v = np.array([[0],[1]])
hh = np.array([[1],[0],[0],[0]])
vv = np.array([[0],[0],[0],[1]])
hv = np.array([[0],[1],[0],[0]])
vh = np.array([[0],[0],[1],[0]])
phi_p = np.array([[1],[0],[0],[1]]/np.sqrt(2));
phi_m = np.array([[1],[0],[0],[-1]]/np.sqrt(2));
psi_p = np.array([[0],[1],[1],[0]]/np.sqrt(2));
psi_m = np.array([[0],[1],[-1],[0]]/np.sqrt(2));
#conjugate transpose
h_ct = np.conj(h.T)
v_ct = np.conj(v.T)

#pauli matrices
sx = np.array([[0, 1],[ 1, 0]])
sy = np.array([[0, -1j],[1j, 0]])
sz = np.array([[1, 0],[0, -1]])

# Convert pure vector state into density matrix
def dens(state):
    return state@np.conj(state.T)

# to find measurement settings given measurement operator sigma
def meas(ang,sigma):
    return QWP(ang[1])@HWP(ang[0])@sigma@np.conj((QWP(ang[1])@HWP(ang[0])).T)

# Generate measurement operator for n qubits as tensor product of the same measurement sigma
def meas_multi(n,sigma):
    if n==1:
        return sigma
    else:
        return np.kron(meas_multi(n-1,sigma),sigma)

# GHZ^n state
def GHZ(nqub=4):
    ghz = np.zeros((2**nqub,1))
    ghz[0],ghz[-1] = 1/np.sqrt(2),1/np.sqrt(2)
    return ghz

# Post-selection operation for GHZ^n states
def Ups(nqub=4):
    operator = np.zeros((2**nqub,2**nqub))
    operator[0,0],operator[-1,-1] = 1,1
    return operator

# normalise quantum state
def normalise(state=dens(phi_p)):
    if np.shape(state)[0] == np.shape(state)[1]: #state is density matrix
        return state/np.trace(state)
    #if state is pure (vector)
    elif np.shape(state)[0] != np.shape(state)[1] and np.shape(state)[1] == 1:
        return state/np.linalg.norm(state)
    else: print("Error: Input state doesn't look right!")

# mixed 2-qubit states (phi or psi, plus or minus)
def mix_2q(p, state=dens(phi_p)):
    if np.shape(state)[0] == np.shape(state)[1]: #state is density matrix
        return p*state+(1-p)*np.eye(np.shape(state)[0])/np.shape(state)[0]
    #if state is pure (vector)
    elif np.shape(state)[0] != np.shape(state)[1] and np.shape(state)[1] == 1:
        return p*dens(state)+(1-p)*np.eye(np.shape(state)[0])/np.shape(state)[0]
    else: print("Error: Input state doesn't look right!")

# mixed product state of N phi^+ pairs, each pair with mixing parameter p
def mix_pairs(N,p):
    phiDM = mix_2q(p,phi_p)
    if N==1:
        return phiDM
    else:
        return normalise(np.kron(mix_pairs(N-1,p),phiDM))
    
# function to calculate expectation value from WP measurement settings
#4-qubits
def exp_4q(state, setts):
    #setts should be a list [[HWP_A1,QWP_A1],[HWP_B1,QWP_B1],[HWP_A2,QWP_A2],[HWP_B2,QWP_B2]]
    #check if state is dens matrix
    if np.shape(state)[0] == np.shape(state)[1]: #state is density matrix
        rho = state
    elif np.shape(state)[0] != np.shape(state)[1] and np.shape(state)[1] == 1: #state is pure
        rho = dens(state)
    meas_op = np.kron(QWP(setts[0][1])@HWP(setts[0][0]),
                      np.kron(QWP(setts[1][1])@HWP(setts[1][0]),
                              np.kron(QWP(setts[2][1])@HWP(setts[2][0]),QWP(setts[3][1])@HWP(setts[3][0]))))
    expectation = np.trace(meas_op@rho@meas_op.conj().T@meas_multi(4,sz))
    
    return expectation

### Fidelity

In [3]:
def fidelity(state1,state2):
    if np.shape(state1)[1] == 1 and np.shape(state2)[1] == 1: #both pure
        rho1 = dens(state1)
        rho2 = dens(state2)
    elif np.shape(state1)[1] == np.shape(state1)[0] and np.shape(state2)[1] == 1: #state 1 DM, 2 pure
        rho1 = state1
        rho2 = dens(state2)
    elif np.shape(state2)[1] == np.shape(state2)[0] and np.shape(state1)[1] == 1: #state 2 DM, 1 pure
        rho1 = dens(state1)
        rho2 = state2
    elif np.shape(state1)[1] == np.shape(state1)[0] and np.shape(state2)[1] == np.shape(state2)[0]: #state 1,2 DMs
        rho1 = state1
        rho2 = state2
    else: 
        print("Error: Input state doesn't look right!")
        return
        
    val = (np.trace(sp.linalg.sqrtm(sp.linalg.sqrtm(rho1)@rho2@(sp.linalg.sqrtm(rho1)))))**2
    val = np.real(val)

    return val

In [32]:
state = meas_multi(6,HWP(22.5))@GHZ(6)

i=0
for element in state:
    print(i,element)
    i += 1


0 [0.1767767]
1 [4.16333634e-17]
2 [4.16333634e-17]
3 [0.1767767]
4 [4.16333634e-17]
5 [0.1767767]
6 [0.1767767]
7 [0.]
8 [4.16333634e-17]
9 [0.1767767]
10 [0.1767767]
11 [0.]
12 [0.1767767]
13 [0.]
14 [0.]
15 [0.1767767]
16 [4.16333634e-17]
17 [0.1767767]
18 [0.1767767]
19 [0.]
20 [0.1767767]
21 [0.]
22 [0.]
23 [0.1767767]
24 [0.1767767]
25 [0.]
26 [0.]
27 [0.1767767]
28 [0.]
29 [0.1767767]
30 [0.1767767]
31 [-4.16333634e-17]
32 [4.16333634e-17]
33 [0.1767767]
34 [0.1767767]
35 [0.]
36 [0.1767767]
37 [0.]
38 [0.]
39 [0.1767767]
40 [0.1767767]
41 [0.]
42 [0.]
43 [0.1767767]
44 [0.]
45 [0.1767767]
46 [0.1767767]
47 [-4.16333634e-17]
48 [0.1767767]
49 [0.]
50 [0.]
51 [0.1767767]
52 [0.]
53 [0.1767767]
54 [0.1767767]
55 [-4.16333634e-17]
56 [0.]
57 [0.1767767]
58 [0.1767767]
59 [-4.16333634e-17]
60 [0.1767767]
61 [-4.16333634e-17]
62 [-4.16333634e-17]
63 [0.1767767]


In [58]:
np.trace(meas_multi(2,sy)@dens(phi_p))
np.kron(QWP(45)@HWP(0),QWP(45)@HWP(0))@phi_p

#CHSH
state = phi_p
np.trace(np.kron(sz,(sz+sx)/np.sqrt(2))@dens(state)-np.kron(sz,(-sz+sx)/np.sqrt(2))@dens(state)+
         np.kron(sx,(sz+sx)/np.sqrt(2))@dens(state)+np.kron(sx,(-sz+sx)/np.sqrt(2))@dens(state))

(np.trace(np.kron(QWP(0)@HWP(0),QWP(0)@HWP(11.25))@dens(state)@np.kron((QWP(0)@HWP(0)),(QWP(0)@HWP(11.25))).conj().T@dens(hh)-
        np.kron(QWP(0)@HWP(0),QWP(0)@HWP(11.25))@dens(state)@np.kron((QWP(0)@HWP(0)),(QWP(0)@HWP(11.25))).conj().T@dens(hv)-
        np.kron(QWP(0)@HWP(0),QWP(0)@HWP(11.25))@dens(state)@np.kron((QWP(0)@HWP(0)),(QWP(0)@HWP(11.25))).conj().T@dens(vh)+
        np.kron(QWP(0)@HWP(0),QWP(0)@HWP(11.25))@dens(state)@np.kron((QWP(0)@HWP(0)),(QWP(0)@HWP(11.25))).conj().T@dens(vv))-
np.trace(np.kron(QWP(0)@HWP(0),QWP(0)@HWP(33.75))@dens(state)@np.kron((QWP(0)@HWP(0)),(QWP(0)@HWP(33.75))).conj().T@dens(hh)-
        np.kron(QWP(0)@HWP(0),QWP(0)@HWP(33.75))@dens(state)@np.kron((QWP(0)@HWP(0)),(QWP(0)@HWP(33.75))).conj().T@dens(hv)-
        np.kron(QWP(0)@HWP(0),QWP(0)@HWP(33.75))@dens(state)@np.kron((QWP(0)@HWP(0)),(QWP(0)@HWP(33.75))).conj().T@dens(vh)+
        np.kron(QWP(0)@HWP(0),QWP(0)@HWP(33.75))@dens(state)@np.kron((QWP(0)@HWP(0)),(QWP(0)@HWP(33.75))).conj().T@dens(vv))+
np.trace(np.kron(QWP(0)@HWP(22.5),QWP(0)@HWP(11.25))@dens(state)@np.kron((QWP(0)@HWP(22.5)),(QWP(0)@HWP(11.25))).conj().T@dens(hh)-
        np.kron(QWP(0)@HWP(22.5),QWP(0)@HWP(11.25))@dens(state)@np.kron((QWP(0)@HWP(22.5)),(QWP(0)@HWP(11.25))).conj().T@dens(hv)-
        np.kron(QWP(0)@HWP(22.5),QWP(0)@HWP(11.25))@dens(state)@np.kron((QWP(0)@HWP(22.5)),(QWP(0)@HWP(11.25))).conj().T@dens(vh)+
        np.kron(QWP(0)@HWP(22.5),QWP(0)@HWP(11.25))@dens(state)@np.kron((QWP(0)@HWP(22.5)),(QWP(0)@HWP(11.25))).conj().T@dens(vv))+
np.trace(np.kron(QWP(0)@HWP(22.5),QWP(0)@HWP(33.75))@dens(state)@np.kron((QWP(0)@HWP(22.5)),(QWP(0)@HWP(33.75))).conj().T@dens(hh)-
        np.kron(QWP(0)@HWP(22.5),QWP(0)@HWP(33.75))@dens(state)@np.kron((QWP(0)@HWP(22.5)),(QWP(0)@HWP(33.75))).conj().T@dens(hv)-
        np.kron(QWP(0)@HWP(22.5),QWP(0)@HWP(33.75))@dens(state)@np.kron((QWP(0)@HWP(22.5)),(QWP(0)@HWP(33.75))).conj().T@dens(vh)+
        np.kron(QWP(0)@HWP(22.5),QWP(0)@HWP(33.75))@dens(state)@np.kron((QWP(0)@HWP(22.5)),(QWP(0)@HWP(33.75))).conj().T@dens(vv)))

print(np.trace(np.kron(sy,np.kron((sx-sy)/np.sqrt(2),np.kron((sx-sy)/np.sqrt(2),(sx+sy)/np.sqrt(2))))@dens(GHZ(4))))

print(np.trace(np.kron(sy,np.kron(sy,np.kron(sy,sy)))@dens(GHZ(4))))

np.trace(np.kron(-sy,np.kron(-sy,np.kron(-sy,sy)))@dens(GHZ(4)))

(0.7071067811865472+0j)
(0.9999999999999998+0j)


(-0.9999999999999998+0j)

In [114]:
print(QWP(22.5)@HWP(-11.25)@(v+np.exp(-1j*np.pi/4)*h)/np.sqrt(2))
QWP(22.5)@HWP(33.75)@(v+np.exp(1j*np.pi/4)*h)/np.sqrt(2)

v1 = (v+np.exp(1j*np.pi/4)*h)/np.sqrt(2)
v2 = (v-np.exp(1j*np.pi/4)*h)/np.sqrt(2)

D = sz
#print(np.shape(v1))
e_mat1 = np.column_stack((v1,v2))

mat1 = e_mat1@sz@np.linalg.inv(e_mat1)

print(mat1)
print((sx-sy)/np.sqrt(2))

[[ 0.        +3.92523115e-17j]
 [-0.38268343-9.23879533e-01j]]
[[0.        +0.j         0.70710678+0.70710678j]
 [0.70710678-0.70710678j 0.        +0.j        ]]
[[0.        +0.j         0.70710678+0.70710678j]
 [0.70710678-0.70710678j 0.        +0.j        ]]


In [55]:
HWP(0)@(h+v)/np.sqrt(2)
A = QWP(-45)@HWP(0)@(h)

print(A)
np.abs(A)

np.trace(sz@dens(A)@dens(h))

np.trace(sz@dens(A))

[[ 0.5+0.5j]
 [-0.5+0.5j]]


0j

In [22]:
settings = [[22.5,0],[33.75,22.5],[33.75,22.5],[33.75,22.5]]

#settings = [[0,45],[0,45],[0,45],[0,-45]]

#settings = [[-22.5,0],[22.5,0],[22.5,0],[22.5,0]]

exp_4q(GHZ(4),settings)

(-0.7071067811865477+0j)